## Install Lib API

In [ ]:
! pip install https://dnaink.jfrog.io/artifactory/dna-ink-pypi/model-fkeywords/0.1.0/model_fkeywords-0.1.0-py3-none-any.whl

In [ ]:
! python -m spacy download pt_core_news_sm

## Import libs

In [1]:
import pandas as pd
from api_model.nlsuper import NlExtractorProcess
from api_model.nlvisualization import NlVisualization

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_10686/1571986624.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Variables

##### Description of process
        whats_process = 'complete'
            return: process all pipeline
        whats_process = 'partial'
            return: findkeywords and process bigrams
        whats_process = 'only_keywords'
            return: findkeywords   

In [2]:
filename = 'LIBERACAO_MARGEM_TESTE'
prefix = 'csv'
prefix_sep = '|'
column_text = 'TRANSCRICAO'
whats_process = 'complete'
#whats_process = 'partial'
#whats_process = 'only_keywords'
id_database = 'ID'
type_find = 'aproximado' # aproximado # fixo
activate_stopwords = 'sim' # não

In [3]:
text_finds = {
    'negatividade': ['a desejar', 'abrir uma reclamacao', 'absurd', 'aguento mais', 'boceta', 'bosta', 'brigaram', 'buceta', 'burocra', 'cansad', 'caralho', 'conflito', 'constrang', 'decepcao', 'decepcionad', 'quanta demora', 'que demora', 'tentando resolver', 'descontent', 'descrenca', 'descrente', 'desrespeit', 'fdp', 'filha da puta', 'filha de uma puta', 'filho da puta', 'filho de uma puta', 'frustracao', 'frustrad', 'humilhacao', 'humilhad', 'ignorad', 'ignoram', 'incompet', 'inferno', 'informa nada', 'injuria', 'ironia', 'ironic', 'irreponsaveis', 'irresponsabilidade', 'irresponsavel', 'ma vontade', 'mal a pior', 'mal atendid', 'mal educad', 'mal respondem', 'mal tratad', 'mau atendid', 'mau educad', 'mau respondem', 'mau tratad', 'merda', 'nao podemos arcar', 'nao posso arcar', 'nao sabe', 'ninguem resolve', 'ofenderam', 'ofendid', 'orrivel', 'pelo amor de deus', 'pessimo', 'pessimu', 'porra', 'poxa', 'pqp', 'puta que pariu', 'reclamacao', 'reclamar', 'ridicul', 'ruim', 'sabe nada', 'sem educacao', 'ta dificil', 'trata mal', 'trata mau', 'tratou mal', 'tratou mau', 'triste', 'vergonh', 'vagabund', 'raiva', 'detest', 'nao quero mais', 'odeio', 'safad', 'deus me livre', 'vergonh'],
    'reputacao': ['advogad', 'consumidor.gov', 'entrar com processo', 'facebook', 'instagram', 'judiciais', 'judicial', 'justica', 'meu direito', 'meus direito', 'ouvidoria', 'pequenas causas', 'processar voces', 'procon', 'reclame aqui', 'reclameaqui', 'twitter'],
    'rechamada': ['ainda nao', 'alguma posicao', 'ate agora', 'ate o momento', 'atraso', 'chamado aberto', 'consigo resolver', 'contato novamente', 'continuo sem acesso', 'de novo', 'demora', 'desde ontem', 'desde semana', 'diversas vezes', 'duas vezes', 'em andamento', 'entrei em contato', 'era pra ter', 'faz um mes', 'faz uma semana', 'ja abri', 'ja enviei', 'ja liguei', 'ja mandei', 'liguei pra central', 'mesmo erro', 'muitas vezes', 'nao chegou', 'nao e a primeira vez', 'ocorrencia aberta', 'posicionamento', 'segunda vez', 'sem sucesso', 'tentativa', 'terceira vez', 'todo dia', 'varias vezes', 'chamado aberto'],
    'satisfacao': ['adorei', 'adoro', 'ageis', 'agil', 'agilidade', 'amei', 'atendimento perfeito', 'bem atendid', 'carisma', 'diferenciad', 'educad', 'eficacia', 'eficas', 'eficaz', 'elogiar', 'elogio', 'excelente atendimento', 'feliz', 'gostei', 'impecavel', 'me ajudou', 'merito', 'motivacao', 'motivad', 'muito 10', 'muito bom', 'muito dez', 'nota 10', 'nota dez', 'otimo atendimento', 'parabenizar', 'parabens', 'pela ajuda', 'personalizad', 'preparad', 'prestativ', 'profissional', 'qualidade', 'rapidez', 'resolveu', 'satisfacao', 'satisfatori', 'satisfeit', 'solucionou', 'ate que enfim', 'ate que em fim'],
    'concorrencia': ['ifood', 'caju', 'vee', 'flash', 'bem', 'up', 'VR', 'ticket', 'alelo', 'swile'],
    'contencao': ['desculpa', 'desculpe', 'sinto muito', 'desculpe a demora', 'lamento', 'lamentamos', 'perdoa', 'perdao', 'pelo ocorrido', 'o ocorrido', 'fique tranquil'],
    'ocorrencia': ['chamado', 'ocorrencia', 'registrada com sucesso'],
    'direcionamento': ['central de atedimento', 'ligar na central', 'somente na central', 'sodexo.com', 'ligue na central'],
    'pedefacil': ['clique no', 'clique em', 'menu servicos', 'do menu', 'pede facil', 'clica no', 'clica em', 'diretamente no site', 'visualizar em', 'no menu', 'menu'],
    'ativo': ['analista pode entrar em contato', 'o analista entrara em contato', 'para te auxiliar na proposta', 'solicitei o contato', 'deixe seu contato']

          }

In [4]:
additional_stop_words = ['porque','bom','dia','tres','três','alo','alô']

In [5]:
interlocutor = {'CANAL': ['LEFT', 'RIGHT']}
response_time = 'STARTTIME'
format_data = '%d/%m/%Y %H:%M:%S|%d/%m/%Y %H:%M|%Y-%m-%d %H:%M:%S|%d-%m-%Y %H:%M|%d%m%Y %H:%M:%S|%d%b%Y:%H:%M:%S' # 03MAR2022:12:01:33

In [6]:
df = NlExtractorProcess.call_process(filename, prefix, prefix_sep,\
                                     column_text, whats_process,\
                                     text_finds, id_database, type_find,\
                                     additional_stop_words, activate_stopwords,\
                                    interlocutor, response_time, format_data)

[06/20 22:13:50] - [Info]:  Load CSV
[06/20 22:13:50] - [Info]:  read file /opt/dna/find-keywords/datalake/csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5002 entries, 0 to 5001
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CPF           5002 non-null   int64  
 1   DURATION      5002 non-null   int64  
 2   ENDTIME       5002 non-null   object 
 3   EXTERNALID    5002 non-null   object 
 4   ID            5002 non-null   int64  
 5   STARTTIME     5002 non-null   object 
 6   MONOCHANNEL   4 non-null      object 
 7   SEQUENCIA     4997 non-null   float64
 8   CANAL         4997 non-null   object 
 9   TRANSCRICAO   4997 non-null   object 
 10  AGENT         5002 non-null   object 
 11  WORKGROUP     5002 non-null   object 
 12  CATEGORIES    5002 non-null   object 
 13  TOPICS        4964 non-null   object 
 14  RECHAMADAS    0 non-null      float64
 15  URA_LOG       5002 non-null   int64  
 16  FIL

22/06/21 01:14:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[06/20 22:15:05] - [Info]:  Finishing Process
[06/20 22:15:05] - [Info]:  created message order
Generating `message_order` column
root
 |-- issue_id: string (nullable = true)
 |-- message_time: string (nullable = true)
 |-- message_author: string (nullable = true)
 |-- message_content: string (nullable = true)
 |-- negatividade_findint: string (nullable = true)
 |-- reputacao_findint: string (nullable = true)
 |-- rechamada_findint: string (nullable = true)
 |-- satisfacao_findint: string (nullable = true)
 |-- concorrencia_findint: string (nullable = true)
 |-- contencao_findint: string (nullable = true)
 |-- ocorrencia_findint: string (nullable = true)
 |-- direcionamento_findint: string (nullable = true)
 |-- pedefacil_findint: string (nullable = true)
 |-- ativo_findint: string (nullable = true)
 |-- message_order: integer (nullable = true)

[06/20 22:15:05] - [Debug]:  print new columns of pyspark dataframe: None
[06/20 22:15:05] - [Info]:  remove null values of dataset


[06/20 22:15:09] - [Debug]:  count rows after remove null values 4315
[06/20 22:15:09] - [Debug]:  created a new collect dict of interlocutor
[06/20 22:15:09] - [Debug]:  new collect dict {'message_author': ['LEFT', 'RIGHT']}
[06/20 22:15:09] - [Debug]:  created a original column
[06/20 22:15:09] - [Info]:  agroup all menssages for ticket
Grouping messages into one unique record
[06/20 22:15:09] - [Debug]:  Message Author Column: message_author
[06/20 22:15:09] - [Debug]:  Message content Column: message_content
[06/20 22:15:09] - [Debug]:  [], []
[06/20 22:15:09] - [Debug]:  separadores cliente e operador, {'message_author': ['LEFT', 'RIGHT']}
[06/20 22:15:09] - [Debug]:  Joining messages to all_messages
[06/20 22:15:09] - [Debug]:  Joining messages to all_messages_message_author_LEFT
[06/20 22:15:09] - [Debug]:  Joining messages to all_messages_message_author_RIGHT
[06/20 22:15:09] - [Debug]:  MAX columns: {'reputacao_findint', 'ocorrencia_findint', 'negatividade_findint', 'satisfaca

root
 |-- issue_id: string (nullable = true)
 |-- has_attendant: integer (nullable = true)
 |-- message_author: string (nullable = true)
 |-- message_time: string (nullable = true)
 |-- message_order: integer (nullable = true)
 |-- reputacao_findint: string (nullable = true)
 |-- ocorrencia_findint: string (nullable = true)
 |-- negatividade_findint: string (nullable = true)
 |-- satisfacao_findint: string (nullable = true)
 |-- rechamada_findint: string (nullable = true)
 |-- ativo_findint: string (nullable = true)
 |-- direcionamento_findint: string (nullable = true)
 |-- pedefacil_findint: string (nullable = true)
 |-- contencao_findint: string (nullable = true)
 |-- concorrencia_findint: string (nullable = true)
 |-- all_messages: string (nullable = true)
 |-- all_messages_message_author_LEFT: string (nullable = true)
 |-- all_messages_message_author_RIGHT: string (nullable = true)
 |-- original_messages: string (nullable = true)
 |-- countent_word: string (nullable = true)
 |-- co

[06/20 22:17:08] - [Debug]:  dataframe old ['CPF', 'DURATION', 'ENDTIME', 'EXTERNALID', 'ID', 'STARTTIME', 'MONOCHANNEL', 'SEQUENCIA', 'CANAL', 'TRANSCRICAO', 'AGENT', 'WORKGROUP', 'CATEGORIES', 'TOPICS', 'RECHAMADAS', 'URA_LOG', 'FILA', 'VQ', 'CALLTYPE', 'CONTACTID', 'DATA_ARQUIVO', 'negatividade_findint', 'reputacao_findint', 'rechamada_findint', 'satisfacao_findint', 'concorrencia_findint', 'contencao_findint', 'ocorrencia_findint', 'direcionamento_findint', 'pedefacil_findint', 'ativo_findint', 'numbers_words']
[06/20 22:17:08] - [Debug]:  dataframe new ['issue_id', 'has_attendant', 'message_author', 'message_time', 'message_order', 'reputacao_findint', 'ocorrencia_findint', 'negatividade_findint', 'satisfacao_findint', 'rechamada_findint', 'ativo_findint', 'direcionamento_findint', 'pedefacil_findint', 'contencao_findint', 'concorrencia_findint', 'all_messages', 'all_messages_message_author_LEFT', 'all_messages_message_author_RIGHT', 'original_messages', 'countent_word', 'countent

In [7]:
df.head()

,CPF,DURATION,ENDTIME,EXTERNALID,ID,STARTTIME,MONOCHANNEL,SEQUENCIA,CANAL,TRANSCRICAO,...,pedefacil_findint,contencao_findint,concorrencia_findint,all_messages,all_messages_message_author_LEFT,all_messages_message_author_RIGHT,original_messages,countent_word,countent_bigram,countent_trigram
0,40361829,39,03MAR2022:11:47:38,DE59V9JKTH2GP6JKP00AEM2EPK029HK4,6837807,None,NaN,0.0,LEFT,,...,None,None,vez,bem vindo cetelem boa nome nome posso ajudar boa tarde boa tarde nome seguinte gostaria saber novo depositar dinheiro pra mim consignado consignado parcela ficou ficou consignado consignado parcela poderia senhora contrato assim parcela descontada ver pra mim sim paguei vinte cinco janeiro ficaram retorno dinheiro dez fevereiro parece pegar outro numero descontando sei conta entendeu nesse caso hoje certo entao vou verificar aqui vou confirmar alguns dados senhora oquei zero dois cinco senhora pode informar cpf cpf quatro zero meia zero zero nome completo nome completo data nascimento data nascimento obrigada dois seis meia dois endereco senhora onde souza cento oitenta apartamento duzentos certo telefone jardim canal cinco cinco nove oito oito oito meia zero nove sete meia sim obrigado confirmacoes momento momento sim cartao enquanto momento momento senhor pode confirmar banco agencia conta pra ser zero sete oito vou pegar cartao pro vinte pera sim zero sete oito zero sete oito agencia conta zero zero dois mil sete digito nove certo banco data brasil posso certo conta corrente poupanca corrente conta constando tava obrigada paulo certo realmente aberto protocolo devolucao sim sido cancelado setor interno aqui banco resposta nesse protocolo sim certo reabrindo protocolo pra devolucao conversei sexta feira moca falou ser depositado sessao sei agora passada falou depositado hoje gostaria saber entao resposta desse protocolo sido aberto abrindo agora sim acionamento segunda vez entao vai ter prazo nada senhora humm valor valor quarenta oito horas uteis pra senhora receber entao humm nove quarta feira senhora recebe valor valor conta parte tarde fim pra depositando realmente certo valor pendente aqui pra gente devolve pra senhora quinhentos noventa cinquenta humm certo centavos confirmei dados bancarios protocolo correto senhora passou certo aham aguardar quarenta oito horas uteis pra receber valor cai conta senhora senhora pode aham varias vezes ligando aqui novamente ligado ligado vez pode olhar vai depositando aproveitando tava achando estranho ser culpa proprio banco sistema sim vamos perceber dessa vez aberto entao ontem agora abri vai passar novo pro setor interno banco humm vai ter resposta semana sim sim depositar faz senhora ligando aqui novamente novo ligando aqui novamente novo certo entao brigada viu banco cetelem banco cetelem agradece otimo brigado tchau tchau,bem vindo cetelem boa nome nome posso ajudar senhora contrato assim parcela descontada certo entao vou verificar aqui vou confirmar alguns dados senhora senhora pode informar cpf cpf nome completo nome completo data nascimento data nascimento endereco senhora certo telefone sim obrigado confirmacoes momento momento momento momento senhor pode confirmar banco agencia conta pra ser sim zero sete oito certo banco certo conta corrente poupanca certo realmente aberto protocolo devolucao sido cancelado setor interno aqui banco resposta nesse protocolo reabrindo protocolo pra devolucao entao resposta desse protocolo sido aberto abrindo agora acionamento segunda vez entao vai ter prazo nada senhora valor valor quarenta oito horas uteis pra senhora receber entao nove quarta feira senhora recebe valor valor conta parte tarde fim pra depositando realmente valor pendente aqui pra gente devolve pra senhora quinhentos noventa cinquenta centavos confirmei dados bancarios protocolo correto senhora passou aguardar quarenta oito horas uteis pra receber valor cai conta senhora senhora pode ligando aqui novamente sim vamos perceber dessa vez aberto ontem agora abri vai passar novo pro 

In [8]:
filename = 'LIBERACAO_MARGEM_TESTE'
column_filter = '' # deixar esse campo vazio caso queira ver tudo
column_text = 'TRANSCRICAO'
whats_process = 'trigram' # trigram or bigram

In [9]:
NlVisualization.wordCloud_Topics(filename,column_filter,column_text,whats_process)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive//LIBERACAO_MARGEM_TESTE_tratado.csv'

In [ ]:
NlVisualization.plot_10_most_common_words(filename,column_filter,column_text)